# Stage 1

Grab the main table of contents and each volumes table of contents


In [ ]:
from bs4 import BeautifulSoup
import os, requests
import time
count = 0
########################################################################################################################
########################################################################################################################
#First phase where we grab the top level table of contents of all volumes, the hyperlink TO each file and the name of the file
########################################################################################################################
########################################################################################################################


# 1 If the 'final' index exists remove it - this is so can re-run script if make changes
if os.path.isfile('index.html') is True: os.remove('index.html')
# 2 If working cache index  doesnt exist then create it
if os.path.isfile('cache') is False:
    print ('main index cache file doesnt exist so grabbing it')
    page = (requests.get('http://edwards.yale.edu/research/browse'))
    soup = BeautifulSoup(page.text, "html.parser")
    with open("cache", "w") as myfile:
        myfile.write(str(soup))
# 3 Otherwise if it does exist lets process it through soup
else:
    print ('main index cache file was found so using' )
    with open('cache', 'r') as f:
        contents = f.read()
    soup = BeautifulSoup(contents, "html.parser")
# 4 Obtain all the A HREF links from the index and populate filename /dirname

for links in soup.findAll('a'):

    filename = os.path.basename(links.get('href'))
    pathname = os.path.dirname(links.get('href'))
#5  Figure out the base url,  update index links to append html and strip base url.
    #this also filters out the urls we dont want from all the hyperlinks found on page
    if "archive" in pathname:
        basepath = pathname
        count = count + 1
        print ('Renaming link ' + str(count))
        links['href'] = links['href'].replace(filename, filename + ".html")
        links['href'] = links['href'].replace(pathname + '/', '')

#6  Remove the 'final' index file if it exists before outputing soup to index
if os.path.isfile('index.html') is True:
    os.remove('index.html')

##
result = soup.select('style, #center ul ')
for items in result:
    with open("index.html", "a") as myfile: myfile.write(str(items))

print ('Finished creating index.html file')

#7  Determine all the urls, filenames  for next part of downloading and output to a dictionary called secondphaselinksandfilenames
   # The select statement homes in on the elements we want from the raw html

secondphaselinksandfilenames = {}
selectquery1 = soup.select('#center ul ')
count = 0
for link in selectquery1:

    raw = link.select('a')
    for item in raw:


        fullfilename =  (item.get('href'))
        nameoffile = fullfilename
        nameoffile = nameoffile.replace('.html', '')
        fullurl = 'http://edwards.yale.edu' + basepath + '/' + nameoffile

        #print (fullfilename)
        #print (fullurl)


        secondphaselinksandfilenames[fullurl] = fullfilename


for key,val in secondphaselinksandfilenames.items():
    print (key)
    print (val)

import csv


with open('stage1.csv', 'w') as f:
    for key in secondphaselinksandfilenames.keys():
        f.write("%s,%s\n" % (key, secondphaselinksandfilenames[key]))

# Stage 2

From each volumes table of contents create a list of all files for all volumes

In [ ]:
from bs4 import BeautifulSoup
import os
import requests
import csv
import time
with open('stage1.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}

#

secondphaselinksandfilenames = {}
for url,filename in mydict.items():

    if os.path.isfile(filename) is False:

        page = (requests.get(url))
        soup = BeautifulSoup(page.text, "html.parser")

        with open(filename, "w") as myfile:
            myfile.write(str(soup))
    with open(filename,'r') as f:
        contents = f.read()

    soup = BeautifulSoup(contents, "html.parser")
    for unwanted in soup("input"):
        unwanted.decompose()
    soup = soup.select('.navlevel1, .navlevel2')
    if os.path.isfile(filename) is False:
        os.remove(filename)
        for items in soup:
            with open(filename, "a") as myfile: myfile.write(str(items))

    for links in soup:
        test =  (links.find('a').attrs)
        filename = (os.path.basename(test.get('href')))
        path = ((os.path.dirname(test.get('href'))))
        fullurl = 'http://edwards.yale.edu/archive' + path + '/' + filename
        fullfilename = filename + '.html'
        secondphaselinksandfilenames[fullurl] = fullfilename

for key,val in secondphaselinksandfilenames.items():
    print (key)
    print (val)

import csv


with open('stage2.csv', 'w') as f:
    for key in secondphaselinksandfilenames.keys():
        f.write("%s,%s\n" % (key, secondphaselinksandfilenames[key]))





# Stage 3

Grab all the files and store them in /files


In [20]:
import csv
import requests
from bs4 import BeautifulSoup
with open('stage2.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}

for url,filename in mydict.items():
    page = (requests.get(url))
    soup = BeautifulSoup(page.text, "html.parser")
    with open('./files/' + filename, "w") as myfile:
        myfile.write(str(soup))

# Stage 4

In [201]:
from bs4 import BeautifulSoup
import csv
# FUNCTIONS
kaggledir = '/kaggle/working/files/'

#   1 - import file to soup
def SoupFILEimport(filename):
    with open(kaggledir + filename, 'r') as f: contents = f.read()
    soup = BeautifulSoup(contents, "html.parser")
    return soup
#   2 - import list of file names from a csv for soup importation
def CSVimport (filename):
    mydict={}
    with open(filename, mode='r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]: rows[1] for rows in reader}
    return mydict
#   3 - unwrap tag 
def U1(soup,list):
    for item in list:
        results = soup.find_all(item)
    for result in results:
        result.unwrap()
    return
#   4 - unwrap tag by element value
def U2 (soup,tag,element,value):
    results = (soup.findAll(tag, {element: value}))
    for result in results:
        result.unwrap()
    return
#   5 - decompose tag
def D1(soup,list):
    for item in list:
        results = soup.find_all(item)
    for result in results:
        result.decompose()
    return
#   6 - decompose tag by element value
def D2 (soup,tag,element,value):
    results = (soup.findAll(tag, {element: value}))
    for result in results:
        result.decompose()
    return
#   7 - select content by css selector
def S1 (soup,cssselector):
    soup =soup.select(cssselector)
    return soup
#   8 - replace tag
def R1 (soup,oldtag,newtag):
    for items in soup.find_all(oldtag):
        new_tag = soup.new_tag(newtag)
        new_tag.string = items.getText()
        items.replace_with(new_tag)    
    return 
#   9 - replace tag by element value
def R2 (soup,oldtag,oldelement,oldvalue, newtag):
    for items in soup.find_all(oldtag, {oldelement:oldvalue}):
        new_tag = soup.new_tag(newtag)
        new_tag.string = items.getText()
        items.replace_with(new_tag)
    return
#   10 - Remove a particular element's attribute
def R3 (soup,att):
    results= (soup.find_all(attrs={'xmlns': True}))
    for items in results:
        del items[att]
    return 
# Find tags, classes , attrib and values
def F1(soup,tag):
    result = soup.find_all(tag)
    return result
#   11 search by classname
def F2(soup,classname):
    result = (soup.findAll('span', {'class': classname}))
    return result
#   12 Find all tags which contain a particular attrib value?
def F3 (soup,tag,attrib,value):
    result = (soup.findAll(tag, {attrib: value}))
    return result
#   13 Select and decompose
def SD(soup,cssselector):
    result = soup.select(cssselector)
    for items in result:
        items.decompose()
    return
#   14 - Select and unwrap
def SU(soup,cssselector):
    result = soup.select(cssselector)
    for items in result:
        items.unwrap()
    return
#   15 - make soup usable again after 'select' has broken it

def Soupify(soup):
    soup = BeautifulSoup(str(soup), "html.parser")
    return soup


In [199]:
mydict = CSVimport('stage2.csv')
for url,filename in mydict.items():
    soup = SoupFILEimport(filename)
    soup = (S1(soup,'#content #text'))
    soup = Soupify(soup)
    U1(soup,['div','front'])
    R3(soup, 'xmlns')
    R2(soup,'span','style','font-style:italic','i')
    U2(soup,'span','style','font-variant:small-caps')
    U1(soup,'br')
    D1(soup,'table')
    SD(soup,'table')
    print (soup)
    time.sleep(30)
#     sys.exit("Exit from script")


[<div id="text"><div type="headnote">
<span class="head">
Freedom of the Will
</span>
<p></p>
<div type="copyright">
<span class="head">
Copyright
</span>
<p></p><center> -- iv -- </center><p>
</p><p><i>©1957 by Yale University Press, Inc.</i></p>
<p><i>All rights reserved. This book may not be reproduced, in whole or in part, in any for (except by reviewers for the public press) without written permission from the publishers. Library of Congress card number: 57-6875</i></p>
<p><i>ISBN 0-300-00848-1</i></p>
<p><i>Printed in the United States of America by Sheridan Books, Inc., Chelsea, Michigan.</i></p>
<p><i>15 14 13 12 11 10 9 8</i></p>
<p></p><center> -- v -- </center><p>
<figure><div class="img"><img src="?path=aHR0cDovL2Vkd2FyZHMueWFsZS5lZHUvZWRpdG9yc2NyaXB0cy9nZXRpbWFnZS5waHA/aW1hZ2U9amVjLXlqZTAxLXYuanBn"/></div> </figure></p><div type="editornote">
<span class="head">
GENERAL EDITOR'S NOTE
</span>
<p>WITH PAUL RAMSEY'S edition of <i>A Careful and Strict Enquiry into the Modern P

KeyboardInterrupt: 

# Stage 5

In [28]:
import csv
import time
import sys
import requests
from bs4 import BeautifulSoup
with open('stage2.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}


    
        
#     sys.exit("Exit from script")

[
<span class="head">
Freedom of the Will
</span>
<p></p>

<span class="head">
Copyright
</span>
<p></p><p>
</p><p><span style="font-style:italic">©1957 by Yale University Press, Inc.</span></p>
<p><span style="font-style:italic">All rights reserved. This book may not be reproduced, in whole or in part, in any for (except by reviewers for the public press) without written permission from the publishers. Library of Congress card number: 57-6875</span></p>
<p><span style="font-style:italic"><span style="font-variant:small caps">ISBN</span> 0-300-00848-1</span></p>
<p><span style="font-style:italic">Printed in the United States of America by Sheridan Books, Inc., Chelsea, Michigan.</span></p>
<p><span style="font-style:italic">15 14 13 12 11 10 9 8</span></p>
<p></p><p>
<figure><img src="?path=aHR0cDovL2Vkd2FyZHMueWFsZS5lZHUvZWRpdG9yc2NyaXB0cy9nZXRpbWFnZS5waHA/aW1hZ2U9amVjLXlqZTAxLXYuanBn"/> </figure></p>
<span class="head">
GENERAL EDITOR'S NOTE
</span>
<p><span style="font-variant:small

KeyboardInterrupt: 